In [1]:
# ---------------------------------------------
# a) Import Required Libraries
# ---------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------
# b) Upload / Access the Dataset
# ---------------------------------------------
(x_train, _), (x_test, _) = mnist.load_data()

# Normalize & Flatten
x_train = x_train.astype("float32") / 255.
x_test  = x_test.astype("float32") / 255.

x_train = x_train.reshape(len(x_train), 28*28)
x_test  = x_test.reshape(len(x_test), 28*28)

# ---------------------------------------------
# c) Encoder – Latent Representation
# ---------------------------------------------
encoder = Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu')       # Latent space
])

# ---------------------------------------------
# d) Decoder – Reconstruct Input
# ---------------------------------------------
decoder = Sequential([
    Dense(64, activation='relu', input_shape=(32,)),
    Dense(128, activation='relu'),
    Dense(784, activation='sigmoid')
])

# Build Autoencoder
autoencoder = Sequential([encoder, decoder])

# ---------------------------------------------
# e) Compile the Model
# ---------------------------------------------
autoencoder.compile(optimizer=Adam(), loss='mse', metrics=['mse'])

# Train
history = autoencoder.fit(
    x_train, x_train,
    epochs=5,
    batch_size=256,
    validation_split=0.2
)

# Evaluate
loss, mse = autoencoder.evaluate(x_test, x_test)
print("Reconstruction MSE:", mse)

# ---------------------------------------------
# Simple Anomaly Detection
# (Higher reconstruction error → Anomaly)
# ---------------------------------------------
recon = autoencoder.predict(x_test)
recon_error = np.mean(np.abs(x_test - recon), axis=1)

threshold = np.percentile(recon_error, 95)   # top 5% considered anomaly
print("Threshold:", threshold)

anomalies = recon_error > threshold
print("Anomalies detected:", np.sum(anomalies))


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - loss: 0.1055 - mse: 0.1055 - val_loss: 0.0443 - val_mse: 0.0443
Epoch 2/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0389 - mse: 0.0389 - val_loss: 0.0287 - val_mse: 0.0287
Epoch 3/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0270 - mse: 0.0270 - val_loss: 0.0237 - val_mse: 0.0237
Epoch 4/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.0209 - val_mse: 0.0209
Epoch 5/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0202 - mse: 0.0202 - val_loss: 0.0190 - val_mse: 0.0190
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0188 - mse: 0.0188
Reconstruction MSE: 0.018455222249031067
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Threshold: 0.08296444
Anomalies detected: 500
